#databricks notebook: GOLD - FireRisk Diário (INPE)

In [0]:
# Databricks notebook: GOLD - FireRisk Diário (INPE)
# =========================================================

from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, lit, avg, min as spark_min, max as spark_max, count
)

spark = SparkSession.builder.getOrCreate()

In [0]:
# =========================================================
# 🔹 Parâmetros
# =========================================================

dbutils.widgets.text("catalog", "")
dbutils.widgets.text("schema_silver", "")
dbutils.widgets.text("table_silver", "")
dbutils.widgets.text("schema_gold", "")
dbutils.widgets.text("table_gold", "")
dbutils.widgets.text("data_ref_carga", "")

catalog         = dbutils.widgets.get("catalog")
schema_silver   = dbutils.widgets.get("schema_silver")
table_silver    = dbutils.widgets.get("table_silver")
schema_gold     = dbutils.widgets.get("schema_gold")
table_gold      = dbutils.widgets.get("table_gold")
data_ref_carga  = dbutils.widgets.get("data_ref_carga")

if not data_ref_carga:
    raise ValueError("❌ Parâmetro 'data_ref_carga' não informado")

print("=====================================")
print(f"📅 Data referência: {data_ref_carga}")
print(f"📦 Silver origem : {catalog}.{schema_silver}.{table_silver}")
print(f"💾 Gold destino  : {catalog}.{schema_gold}.{table_gold}")
print("=====================================")

In [0]:
# =========================================================
# 🔹 Leitura da Silver
# =========================================================

df_silver = spark.table(f"{catalog}.{schema_silver}.{table_silver}") \
    .filter(col("data_ref_carga") == data_ref_carga)

total_linhas = df_silver.count()
print(f"🔸 Registros Silver lidos: {total_linhas}")

if total_linhas == 0:
    raise RuntimeError("❌ Silver vazia para esta data. Encerrando job.")

In [0]:
# =========================================================
# 🔹 Agregação GOLD — Resumo Diário Brasil
# =========================================================
df_gold = (
    df_silver
    .groupBy("data_ref_carga")
    .agg(
        avg("rf").alias("rf_medio"),
        spark_min("rf").alias("rf_min"),
        spark_max("rf").alias("rf_max"),
        count("*").alias("qtde_pontos_grade")
    )
    .withColumn("processado_em", lit(current_timestamp()))
)

print("🔸 Gold gerada com sucesso")

In [0]:
# =========================================================
# 🔹 Escrita Gold (Delta)
# =========================================================
(
    df_gold.write
    .format("delta")
    .mode("overwrite")
    .option("replaceWhere", f"data_ref_carga = '{data_ref_carga}'")
    .saveAsTable(f"{catalog}.{schema_gold}.{table_gold}")
)

print(f"💾 Gold salva em: {catalog}.{schema_gold}.{table_gold}")

In [0]:
# =========================================================
# 🔹 Comentários da GOLD
# =========================================================
spark.sql(f"""
COMMENT ON TABLE {catalog}.{schema_gold}.{table_gold} IS
'Tabela GOLD - Resumo diário do risco de fogo INPE FireRisk 2.2 (média, mínimo, máximo, quantidade de pontos).';

COMMENT ON COLUMN {catalog}.{schema_gold}.{table_gold}.data_ref_carga IS 'Data de referência AAAAMMDD';
COMMENT ON COLUMN {catalog}.{schema_gold}.{table_gold}.rf_medio IS 'Valor médio do risco de fogo no dia';
COMMENT ON COLUMN {catalog}.{schema_gold}.{table_gold}.rf_min IS 'Menor valor de risco de fogo no dia';
COMMENT ON COLUMN {catalog}.{schema_gold}.{table_gold}.rf_max IS 'Maior valor de risco de fogo no dia';
COMMENT ON COLUMN {catalog}.{schema_gold}.{table_gold}.qtde_pontos_grade IS 'Quantidade de pixels analisados no grid para o dia';
COMMENT ON COLUMN {catalog}.{schema_gold}.{table_gold}.processado_em IS 'Timestamp de processamento na camada Gold';
""")

print("📝 Comentários adicionados com sucesso!")